# Stochastic Variational Method with RL algorithms

In [1]:
import numpy as np
import gym
import svm_env as svm
import torch
import subprocess

## Expoloring environment

In [2]:
env = gym.make('svm_env:svmEnv-v0', file_sigmas ="./svmCodeSVD/sigmas.dat" )

obs_space = env.observation_space

print('###### Observation space ####### \n', obs_space)

state_size = env.observation_space.shape[-1]

print('###### Size of observation space ####### \n', state_size)

act_space = env.action_space

print('###### Action space ####### \n', act_space)

act_size = env.action_space.shape[-1]

print('###### Number of actions ####### \n', act_size)

state = env.reset()

print('##### State after reset ###### \n', state)

print('##### File where will be stored sigmas \n', env.file_sigmas)

t = act_space.sample()
t

###### Observation space ####### 
 Box(-inf, inf, (1,), float32)
###### Size of observation space ####### 
 1
###### Action space ####### 
 Box(-1.0, 1.0, (3,), float32)
###### Number of actions ####### 
 3
#### CALL RESET ####
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
##### State after reset ###### 
 [0.]
##### File where will be stored sigmas 
 ./svmCodeSVD/sigmas.dat


array([ 0.9306232 ,  0.6120736 , -0.04571519], dtype=float32)

In [ ]:
if (state[0] == 0.0):
    print('**Ciao')

In [ ]:
a1 = act_space.sample()
print(a1)

env.reset()
print(env.sigmas)


env.actions_taken.append(a1)

env.sigmas = open(env.file_sigmas, 'w')

np.savetxt(env.sigmas, env.actions_taken, fmt="%f")

env.sigmas.close()

a2 = act_space.sample()
print(a2)

env.actions_taken.append(a2)

env.sigmas = open(env.file_sigmas, 'w')

np.savetxt(env.sigmas, env.actions_taken, fmt="%f")

env.sigmas.close()

print(env.file_sigmas)

import subprocess

result = subprocess.check_output(['./svmCodeSVD/svmThree', './svmCodeSVD/remmy.input', env.file_sigmas]).splitlines()

print(result)

result = np.array(result,dtype=float)
result_en = result[0]

princp_dim = int(result[1])
full_dim = int(result[2])
print(princp_dim, full_dim, len(env.actions_taken)) 

## DDPG from `stable_baseline3`

In [ ]:
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

# The noise objects for DDPG
action_noise = NormalActionNoise(mean=np.zeros(act_size), sigma=0.1 * np.ones(act_size))

model = DDPG("MlpPolicy", env, action_noise = action_noise, \
            batch_size=128, gamma=1.0, verbose=1)

# (policy, env, learning_rate=0.001, buffer_size=1000000,learning_starts=100, batch_size=100, 
# tau=0.005, gamma=0.99, train_freq=(1, 'episode'),  gradient_steps=- 1, action_noise=None, 
# replay_buffer_class=None, replay_buffer_kwargs=None,  optimize_memory_usage=False, 
# tensorboard_log=None, create_eval_env=False, policy_kwargs=None,  verbose=0, seed=None, 
# device='auto', _init_setup_model=True)

model.learn(total_timesteps=1000, log_interval=5)

# learn(total_timesteps, callback=None, log_interval=4, eval_env=None, eval_freq=- 1,
# n_eval_episodes=5, tb_log_name='DDPG', eval_log_path=None, reset_num_timesteps=True)

## PPO with GAE from `stable_baseline3` 

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

model = PPO("MlpPolicy", env, verbose=1, n_steps = 400)

# classstable_baselines3.ppo.PPO(policy, env, learning_rate=0.0003, n_steps=2048, 
#         batch_size=64, n_epochs=10, gamma=0.99, gae_lambda=0.95, clip_range=0.2, 
#         clip_range_vf=None, ent_coef=0.0, vf_coef=0.5, max_grad_norm=0.5, 
#         use_sde=False, sde_sample_freq=- 1, target_kl=None, tensorboard_log=None, 
#         create_eval_env=False, policy_kwargs=None, verbose=0, seed=None, device='auto', 
#         _init_setup_model=True)

model.learn(total_timesteps = 1000)

# learn(total_timesteps, callback=None, log_interval=1, eval_env=None, eval_freq=- 1, 
#       n_eval_episodes=5, tb_log_name='PPO', eval_log_path=None, reset_num_timesteps=True)

model.save("ppo_svm")

Using cpu device
Wrapping the env in a DummyVecEnv.
#### CALL RESET ####
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
#### CALL STEP ####
Action chosen at step:  [ 28.92881     6.0894737 110.       ]
Basis size (it should be the same of full dim) =   1
With this action the energy is:  0.709763
With this action the full dim is:  1  and princip dim is:  1
#### THE ENERGY IS GREATER THEN THE PREVIOUS ONE #### --> Set reward:  -8.09763
Store the energy got and sigmas!
#### CALL STEP ####
Action chosen at step:  [ 0.      59.73591  0.     ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [ 21.858734  29.405764 110.      ]
Basis size (it should be the same of full dim) =   2
With this action the energy is:  -0.034855
With this action the full dim is:  2  and princip dim is:  2
#### THE ACTION IS A GOOD ONE #### --> Store the ene

With this action the energy is:  -0.0627567
With this action the full dim is:  10  and princip dim is:  10
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is positive! 10.01285
Calculate the diff between dim: 
Diff 2 is:  0
#### CALL STEP ####
Action chosen at step:  [110.        0.       81.05438]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [ 5.6064224 22.199787  17.211151 ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [ 0.       29.324411 61.94048 ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [47.65933  57.549637  0.      ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVE

With this action the energy is:  -0.0876984
With this action the full dim is:  16  and princip dim is:  16
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is positive! 16.052239999999998
Calculate the diff between dim: 
Diff 2 is:  0
#### CALL STEP ####
Action chosen at step:  [61.809902  0.       70.97414 ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [  0.        39.258224 104.28931 ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [110. 110.   0.]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [  8.436375  99.798096 109.13034 ]
Basis size (it should be the same of full dim) =   17
With this action

With this action the energy is:  -0.143896
With this action the full dim is:  26  and princip dim is:  26
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is positive! 26.00104
Calculate the diff between dim: 
Diff 2 is:  0
#### CALL STEP ####
Action chosen at step:  [ 25.41736  39.87423 108.80887]
Basis size (it should be the same of full dim) =   27
With this action the energy is:  -0.143914
With this action the full dim is:  27  and princip dim is:  27
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is positive! 27.004859999999997
Calculate the diff between dim: 
Diff 2 is:  0
#### CALL STEP ####
Action chosen at step:  [87.23604 45.05281 78.89728]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [ 77.531494  36.41444  105.9547  ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and si

With this action the energy is:  -0.146304
With this action the full dim is:  37  and princip dim is:  37
#### THE ENERGY IS GREATER THEN THE PREVIOUS ONE #### --> Set reward:  -1.0
Store the energy got and sigmas!
#### CALL STEP ####
Action chosen at step:  [  0.      103.72279   0.     ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [57.011288 43.81432  42.55438 ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [15.677704 50.65246  42.98219 ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [  0.       66.39885 110.     ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas,

...

With this action the energy is:  -0.149655
With this action the full dim is:  195  and princip dim is:  168
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is positive! 168.01680000000005
Calculate the diff between dim: 
Diff 2 is:  2
Add a small PENALTY on the rewards!!
Reward is slightly negative:  -67.20672000000002
#### CALL STEP ####
Action chosen at step:  [ 77.60808 110.      110.     ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [51.54504  55.092197 90.59973 ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [66.17936   8.129768 79.00102 ]
Basis size (it should be the same of full dim) =   196
With this action the energy is:  -0.149655
With this action the full dim is:  196  and princip dim is:  169
#### THE ENERG

With this action the energy is:  -0.149646
With this action the full dim is:  207  and princip dim is:  176
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is positive! 176.00176000000002
Calculate the diff between dim: 
Diff 2 is:  1
Add a small PENALTY on the rewards!!
Reward is slightly negative:  -35.200352
#### CALL STEP ####
Action chosen at step:  [ 47.535343    1.4769783 110.       ]
Basis size (it should be the same of full dim) =   208
With this action the energy is:  -0.149646
With this action the full dim is:  208  and princip dim is:  177
#### THE ENERGY IS GREATER THEN THE PREVIOUS ONE #### --> Set reward:  -1.0
Store the energy got and sigmas!
#### CALL STEP ####
Action chosen at step:  [72.909096 70.06908  37.61917 ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [ 52.414494 110.       110.      ]
**** ILLEGAL ACTION **** --> Se

With this action the energy is:  -0.149641
With this action the full dim is:  212  and princip dim is:  178
#### THE ENERGY IS GREATER THEN THE PREVIOUS ONE #### --> Set reward:  -1.00003
Store the energy got and sigmas!
#### CALL STEP ####
Action chosen at step:  [94.41521  94.43801  85.993744]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [ 0.36003494  0.         57.601097  ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [39.50439  53.75067  62.442524]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [  0.   0. 110.]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, th

With this action the energy is:  -0.149634
With this action the full dim is:  221  and princip dim is:  180
#### THE ENERGY IS GREATER THEN THE PREVIOUS ONE #### --> Set reward:  -1.00004
Store the energy got and sigmas!
#### CALL STEP ####
Action chosen at step:  [ 0.       0.      72.43876]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [ 64.772606 110.       110.      ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [ 0.       0.      50.46089]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [105.63199    5.541088   0.      ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and s

With this action the energy is:  -0.149632
With this action the full dim is:  230  and princip dim is:  183
#### THE ENERGY IS GREATER THEN THE PREVIOUS ONE #### --> Set reward:  -1.0
Store the energy got and sigmas!
#### CALL STEP ####
Action chosen at step:  [  0.        32.584652 110.      ]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [89.9527    0.       24.339607]
**** ILLEGAL ACTION **** --> Set reward: -10.0
This action IS REMOVED from actions taken and sigmas, the energy is NOT STORED!
#### CALL STEP ####
Action chosen at step:  [34.26177  20.325157 71.969986]
Basis size (it should be the same of full dim) =   231
With this action the energy is:  -0.149632
With this action the full dim is:  231  and princip dim is:  183
#### THE ENERGY IS GREATER THEN THE PREVIOUS ONE #### --> Set reward:  -1.0
Store the energy got and sigmas!
#### CALL STEP ####
Action

## From my `ddpg_agent.py` code

In [ ]:
from ddpg_agent import Agent

In [ ]:
env = gym.make('svm_env:svmEnv-v0')
# Instance of the ddpg agent
agent = Agent(1, 3, random_seed=2)

### Training loop
def run_ddpg(max_t_step = 500, n_episodes = 1000):        
    """Deep Deterministic Policy Gradient learning for Reacher Unity Environment.
    
    Params Input
    ==========
        n_episode (int): maximum number of episodes
        queue (int): number of consecutive episodes 
        
    Params Output
    ==========
        scores_all (list of floats): are the scores collected at the end of each episode
        
    """
    
    ##Inizialization
    scores = []                         
    last_energies = []
    princip_dim = []
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        agent.reset()                  
        score = 0.0
               
        ## Training loop of each episode
        for t_step in range(max_t_step):
            action = agent.act(state)
            next_state, reward, done, info = env.step(action)                   
            agent.step(state, action, reward, next_state, done)
            score += reward
            state = next_state  
            if done:                                  
                break
        
        scores.append(score)
        last_energies.append(state[0])
        princip_dim.append(env.princp_dim)
        
        print('Episode {} ... Reward: {:.3f}'.format(i_episode, score))

    return scores, last_energies, princip_dim

In [ ]:
scores, energies, princip_dim = run_ddpg()
torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')

In [ ]:
len(env.actions_taken)
print(env.file_sigmas)

In [ ]:
import matplotlib.pyplot as plt
scores = np.loadtxt('scores_RL_tri_0.out')
energies = np.loadtxt('energies_RL_tri_0.out')
dim = np.loadtxt('princip_dim_tri_0.out')

plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')

In [ ]:
plt.plot(np.arange(len(energies)), energies)
plt.ylabel('Eenergies (mK)')
plt.xlabel('Episode #')

In [ ]:
plt.plot(np.arange(len(dim)), dim)
plt.ylabel('dim (mK)')
plt.xlabel('Episode #')

## Random search as in original SVM

In [ ]:
state = env.reset()
scores = []
step = 0
score = 0.0

while True:
    print(".....STEP.....", step)
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action)
    step = step + 1
    score += reward
    scores.append(score)
    state = next_state
    if done:
        break

In [ ]:
for i in range(10):
    if i == 2:
        pass
    else:
        print(i)